In [1]:
import os
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from itertools import chain
from pyspark.sql.functions import udf

In [2]:
DATAFOLDER = '/Users/christian/Data/udacity_capstone/'

In [3]:
def get_immi_schema():
    """
    Map the column names to datatypes and return as a schema.
    """
    immi_schema = T.StructType([
        T.StructField('cicid', T.IntegerType()),
        T.StructField('i94yr', T.StringType()),
        T.StructField('i94mon', T.StringType()),
        T.StructField('i94cit', T.StringType()),
        T.StructField('i94res', T.IntegerType()),
        T.StructField('i94port', T.StringType()),
        T.StructField('arrdate', T.StringType()),
        T.StructField('i94mode', T.StringType()),
        T.StructField('i94addr', T.StringType()),
        T.StructField('depdate', T.StringType()),
        T.StructField('i94bir', T.StringType()),
        T.StructField('i94visa', T.StringType()),
        T.StructField('count',  T.IntegerType()),
        T.StructField('dtadfile', T.StringType()),
        T.StructField('visapost', T.IntegerType()),
        T.StructField('occup', T.StringType()),
        T.StructField('entdepa', T.StringType()),
        T.StructField('entdepd', T.IntegerType()),
        T.StructField('entdepu', T.StringType()),
        T.StructField('matflag', T.StringType()),
        T.StructField('biryear', T.StringType()),
        T.StructField('dtaddto', T.StringType()),
        T.StructField('gender', T.StringType()),
        T.StructField('insnum', T.StringType()),
        T.StructField('airline', T.StringType()),
        T.StructField('admnum', T.StringType()),
        T.StructField('fltno', T.StringType()),
        T.StructField('visatype', T.StringType()),
    ])
    return immi_schema


In [4]:
schema = get_immi_schema()


In [5]:

spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
#df_spark = spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
df_spark = spark.read.option("mergeSchema", "true").parquet(os.path.join(DATAFOLDER, 'sas_data'))

# convert data types

In [6]:
df_spark = df_spark.withColumn('i94yr', df_spark['i94yr'].cast(T.IntegerType())).\
        withColumn('i94mon', df_spark['i94mon'].cast(T.IntegerType())).\
        withColumn('i94cit', df_spark['i94cit'].cast(T.IntegerType())).\
        withColumn('i94res', df_spark['i94res'].cast(T.IntegerType())).\
        withColumn('arrdate', df_spark['arrdate'].cast(T.IntegerType())).\
        withColumn('i94mode', df_spark['i94mode'].cast(T.IntegerType())).\
        withColumn('depdate', df_spark['depdate'].cast(T.IntegerType())).\
        withColumn('i94bir', df_spark['i94bir'].cast(T.IntegerType())).\
        withColumn('i94visa', df_spark['i94visa'].cast(T.IntegerType())).\
        withColumn('count', df_spark['count'].cast(T.IntegerType())).\
        withColumn('biryear', df_spark['biryear'].cast(T.IntegerType())).\
        withColumn('admnum', df_spark['admnum'].cast(T.IntegerType()))

In [13]:
df_sample = pd.read_csv(os.path.join(DATAFOLDER, 'immigration_data_sample.csv'))

In [14]:
df_spark.head()

Row(cicid=459651.0, i94yr=2016, i94mon=4, i94cit=135, i94res=135, i94port='ATL', arrdate=20547, i94mode=1, i94addr='FL', depdate=20559, i94bir=54, i94visa=2, count=1, dtadfile='20160403', visapost=None, occup=None, entdepa='O', entdepd='R', entdepu=None, matflag='M', biryear=1962, dtaddto='07012016', gender=None, insnum=None, airline='VS', admnum=2147483647, fltno='00115', visatype='WT')

In [77]:
df_time = df_spark
df_time = df_time.withColumn("arrival_date", F.expr("date_add(to_date('1960-01-01'), arrdate)"))
df_time = df_time.withColumn("depart_date", F.expr("date_add(to_date('1960-01-01'), depdate)"))


In [80]:
df_time.withColumn("diff_days", F.datediff("depart_date", "arrival_date")).show()

+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+-----------+---------+
|   cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|    admnum|fltno|visatype|arrival_date|depart_date|diff_days|
+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+-----------+---------+
|459651.0| 2016|     4|   135|   135|    ATL|  20547|      1|     FL|  20559|    54|      2|    1|20160403|    null| null|      O|      R|   null|      M|   1962|07012016|  null|  null|     VS|2147483647|00115|      WT|  2016-04-03| 20

## map i94addrl - state

In [109]:

def map_col(df, map_col_name, df_col_name, new_col_name):
    """
    
    Parameters
    ----------
    df : spark dataframe
        The file containing the df_col_name to be used for mapping.
    map_col_name : str
        The column name of the mapping file.
    df_col_name : str
        The column name in the Spark dataframe to be used.
    new_col_name : str
        New column name of the mapping results.
    
    
    """
    df_map = pd.read_csv(os.path.join(DATAFOLDER, f'{map_col_name}.csv'), quotechar="'")
    dic_map = dict(zip(df_map['value'], df_map[map_col_name]))
    mapping_expr = F.create_map([F.lit(x) for x in chain(*dic_map.items())])
    return df.withColumn(new_col_name, mapping_expr[F.col(df_col_name)])



@udf
def udf_city_name(city_full):
    splt = str.split(city_full, ',')
    if len(splt) == 0:
        return ''
    return splt[0].capitalize().strip()

@udf
def udf_state_short(city_full):
    splt = str.split(city_full, ',')
    if len(splt) < 2:
        return ''
    return splt[1].strip()

@udf
def udf_state_format(port_state):
    return port_state.capitalize()

In [110]:
df_test.select("i94addr").distinct().show(10)

+-------+
|i94addr|
+-------+
|     CI|
|     FT|
|     SC|
|     AZ|
|     PU|
|     UA|
|     EA|
|     NS|
|     KI|
|     PI|
+-------+
only showing top 10 rows



# TODO replace mapping in i94addrl to have names in captialize!

In [124]:
#df, map_col_name, df_col_name, new_col_name
df_test = df_time
#df_test = map_col(df_test, 'i94prtl', 'i94port', 'port')

#df_test = df_test.withColumn("port_city", udf_city_name("port"))
df_test = map_col(df_test, 'i94prtl_city', 'i94port', 'port_city')
#df_test = df_test.withColumn("port_state_short", udf_state_short("port"))
df_test = map_col(df_test, 'i94prtl_state', 'i94port', 'port_state_short')
df_test = map_col(df_test, 'i94addrl', 'port_state_short', 'port_state')
#df_test = df_test.withColumn('port_state_2', udf_state_format('port_state'))

df_test = map_col(df_test, 'i94cntyl', 'i94cit', 'state_cit')
df_test = map_col(df_test, 'i94cntyl', 'i94res', 'state_res')
#df_test = map_col(df_test, 'i94addrl', 'i94addr', 'port_state')
df_test = map_col(df_test, 'i94visa', 'i94visa', 'visa')
#df_test = df_test.withColumn("port_state", udf_state_format("port_state"))

In [125]:
df_test.where(F.col("i94port") == 'LOS').show()

+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+-----------+-----------+----------------+----------+--------------+--------------+--------+
|   cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|    admnum|fltno|visatype|arrival_date|depart_date|  port_city|port_state_short|port_state|     state_cit|     state_res|    visa|
+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+-----------+-----------+----------------+----------+--------------+--------------+--------+
|467106.0| 2016|     4|   13

In [117]:
df_test.where(F.col("port_state").isNotNull()).show()

+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+-----------+--------------------+---------+----------------+----------+--------------+--------------+--------+
|   cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|    admnum|fltno|visatype|arrival_date|depart_date|                port|port_city|port_state_short|port_state|     state_cit|     state_res|    visa|
+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+------------+-----------+--------------------+---------+----------------+----------+-----------

Is arrdate ever null?

In [118]:
df_test.where(F.col("arrdate").isNull()).show()

+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+-----------+----+---------+----------------+----------+---------+---------+----+
|cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear|dtaddto|gender|insnum|airline|admnum|fltno|visatype|arrival_date|depart_date|port|port_city|port_state_short|port_state|state_cit|state_res|visa|
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+-------+------+------+-------+------+-----+--------+------------+-----------+----+---------+----------------+----------+---------+---------+----+
+-----+-----+------+------+------+-------+-------+-------+-------+-------+------+----

# merge with demographic data

In [119]:
demographic = pd.read_csv(os.path.join(DATAFOLDER, 'us-cities-demographics.csv'), sep=';')

In [120]:
demographic

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402
1,Peoria,Illinois,33.1,56229.0,62432.0,118661,6634.0,7517.0,2.40,IL,American Indian and Alaska Native,1343
2,O'Fallon,Missouri,36.0,41762.0,43270.0,85032,5783.0,3269.0,2.77,MO,Hispanic or Latino,2583
3,Hampton,Virginia,35.5,66214.0,70240.0,136454,19638.0,6204.0,2.48,VA,Black or African-American,70303
4,Lakewood,Colorado,37.7,76013.0,76576.0,152589,9988.0,14169.0,2.29,CO,Hispanic or Latino,33630
...,...,...,...,...,...,...,...,...,...,...,...,...
2886,Allen,Pennsylvania,33.5,60626.0,59581.0,120207,5691.0,19652.0,2.67,PA,American Indian and Alaska Native,1076
2887,German,Maryland,34.9,41115.0,43007.0,84122,2443.0,27877.0,2.95,MD,White,43330
2888,Pittsburg,California,34.5,33309.0,36118.0,69427,2109.0,21043.0,3.15,CA,White,25996
2889,New Bedford,Massachusetts,38.6,43793.0,51166.0,94959,4185.0,19024.0,2.39,MA,Black or African-American,8890


match airport with demographic table

In [ ]:
# look up unique combinations of city and state in fact table
# 

In [ ]:
# subset with unique combinations of city, state and 
# port_city, port_state, i94port (as key )

In [81]:
df_test.show()

+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+-----------+--------------------+---------+----------+--------------+--------------+--------+--------+
|   cicid|i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|    admnum|fltno|visatype|depart_date|                port|port_city|port_state|     state_cit|     state_res|us_state|    visa|
+--------+-----+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+----------+-----+--------+-----------+--------------------+---------+----------+--------------+--------------+--------+--------+
|459651.0| 2016|     4|   135|   1

In [ ]:
df_intermediate = df_test.select()

# create single tables for everything